In [1]:
from sage.coding.linear_code import AbstractLinearCode
from sage.coding.encoder import Encoder
from sage.coding.decoder import Decoder

class Goppa(AbstractLinearCode):
    def __init__(self, generating_pol, defining_set):
        if not generating_pol.is_monic():
            raise ValueError("ERROR. Generating polynomial isn't monic")
        
        for gamma in defining_set:
            if generating_pol(gamma) == 0:
                raise ValueError("ERROR. Defining elements are roots of generating polynomial")
        
        self._field = generating_pol.base_ring().prime_subfield()
        
        if (not self._field.is_field() or not self._field.is_finite()):
            raise ValueError("ERROR. Generating polynomial isn't definied over a finite field")
        
        self._length = len(defining_set)
        self._generating_pol = generating_pol
        self._defining_set = defining_set
        self._field = generating_pol.base_ring().prime_subfield()
        self._dimension = len(defining_set) - generating_pol.degree()
        
        super(Goppa, self).__init__(self._field, self._length, "GoppaEncoder", "GoppaDecoder")
        
    def parity_check_matrix(self):
        g = self._generating_pol
        L = self._defining_set
        n = self._length
        d = g.degree()
        
        aux = vector([0 for i in range(n)])  # auxiliary vector of n columns
        H = matrix(aux)

        for i in range(0, d):
            elem = g(L[0]).inverse_of_unit() * L[0]**i   # first row
            c = vector(elem).column()
            aux = matrix(c)
            
            for j in range(1,n):
                elem = g(L[j]).inverse_of_unit() * L[j]**i
                c = vector(elem).column()
                aux = aux.augment(c)
            
            H = H.stack(aux)

        H = H.delete_rows([0])   # delete auxiliary vector
        
        return H
    
    def _repr_(self):
        return "[{}, {}] Goppa code".format(
                self.length(), self.dimension())
    
    #def encode(m):
    #    return u * self.G
    
class GoppaEncoder(Encoder):
    def __init__(self, code):
        print("TODO")
        
class GoppaDecoder(Decoder):
    def __init__(self, code):
        print("TODO")

Goppa._registered_encoders["GoppaEncoder"] = GoppaEncoder
Goppa._registered_decoders["GoppaDecoder"] = GoppaDecoder

In [2]:
F = GF(2^3)
R.<x> = F[]
g = x^2 + x + 1
L = [a for a in F.list() if g(a) != 0]
C = Goppa(g, L)

In [3]:
C

[8, 6] Goppa code

In [4]:
C.parity_check_matrix()

[1 0 0 0 0 0 0 1]
[0 0 1 0 1 1 1 0]
[0 1 1 1 0 0 1 0]
[0 1 1 1 1 1 1 1]
[0 1 0 1 1 0 1 0]
[0 0 1 1 1 1 0 0]